# Lecture 6

Data Science, Fall 2023



A demonstration of exploratory data analysis to accompany Lecture 6 (originally planned for Lecture 5).

In [1]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 9)

sns.set()
sns.set_context('talk')

np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)
# This option stops scientific notation for pandas
pd.set_option('display.float_format', '{:.2f}'.format)


These options are used to customize the display of pandas DataFrames for better readability and precision when working with data. Here's why each option is helpful:

# Tuberculosis in the United States

What can we say about the presence of Tuberculosis in the United States?

Let's look at the data included in the [original CDC article](https://www.cdc.gov/mmwr/volumes/71/wr/mm7112a1.htm?s_cid=mm7112a1_w#T1_down) published in 2021.

<br/>

---

# CSV and Nice Field Names
Suppose Table 1 was saved as a CSV file located in `data/cdc_tuberculosis.csv`.

We can then explore the CSV (which is a text file, and does not contain binary-encoded data) in many ways:
1. Using a text editor like the one in DataHub (right-click on the file and use `Open->Editor`), emacs, vim, VSCode, etc.
2. Opening the CSV directly in DataHub (read-only), Excel, Google Sheets, etc.
3. The Python file object
4. pandas, using `pd.read_csv()`

1, 2. Let's start with the first two so we really solidify the idea of a CSV as **rectangular data (i.e., tabular data) stored as comma-separated values**.



4. Finally, let's see the tried-and-true Data Science approach: pandas.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [46]:
# Read tuberculosis data
tuberculosis_data=pd.read_csv("/content/cdc_tuberculosis.csv")
tuberculosis_data.head(5)

,Unnamed: 0,No. of TB cases,Unnamed: 2,Unnamed: 3,TB incidence,Unnamed: 5,Unnamed: 6
0,U.S. jurisdiction,2019,2020,2021,2019.00,2020.00,2021.00
1,Total,"8,900","7,173","7,860",2.71,2.16,2.37
2,Alabama,87,72,92,1.77,1.43,1.83
3,Alaska,58,58,58,7.91,7.92,7.92
4,Arizona,183,136,129,2.51,1.89,1.77


Wait, what's up with the "Unnamed" column names? And the first row, for that matter?

Congratulations -- you're ready to wrangle your data. Because of how things are stored, we'll need to clean the data a bit to name our columns better.

A reasonable first step is to identify the row with the right header. The `pd.read_csv()` function ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)) has the convenient `header` parameter:

In [48]:
# Answer Here
tuberculosis_data=pd.read_csv("/content/cdc_tuberculosis.csv", thousands=",",header=1  )
tuberculosis_data.head()


,U.S. jurisdiction,2019,2020,2021,2019.1,2020.1,2021.1
0,Total,8900,7173,7860,2.71,2.16,2.37
1,Alabama,87,72,92,1.77,1.43,1.83
2,Alaska,58,58,58,7.91,7.92,7.92
3,Arizona,183,136,129,2.51,1.89,1.77
4,Arkansas,64,59,69,2.12,1.96,2.28


Wait...but now we can't differentiate betwen the "Number of TB cases" and "TB incidence" year columns. pandas has tried to make our lives easier by automatically adding ".1" to the latter columns, but this doesn't help us as humans understand the data.

We can do this manually with `df.rename()` [documentation]

In [68]:
# Rename column List
new_columns_name={
    "U.S. jurisdiction" : "States",
    "2019" :"TB_cases(2019)",
    "2020":"TB_cases(2020)",
    "2021":"TB_cases(2021)",
    "2019.1" : "TB_inci(2019)",
    "2020.1":"TB_inci(2020)",
    "2021.1" : "TB_inci(2021)"
}
tuberculosis_data.rename(columns=new_columns_name, inplace=True)

tuberculosis_data.head()

,States,TB_cases(2019),TB_cases(2020),TB_cases(2021),TB_inci(2019),TB_inci(2020),TB_inci(2021)
0,Total,8900,7173,7860,2.71,2.16,2.37
1,Alabama,87,72,92,1.77,1.43,1.83
2,Alaska,58,58,58,7.91,7.92,7.92
3,Arizona,183,136,129,2.51,1.89,1.77
4,Arkansas,64,59,69,2.12,1.96,2.28


In [69]:
cols = tuberculosis_data.columns
cols

Index(['States', 'TB_cases(2019)', 'TB_cases(2020)', 'TB_cases(2021)',
       'TB_inci(2019)', 'TB_inci(2020)', 'TB_inci(2021)'],
      dtype='object')

<br/><br/>

---

# Record Granularity

You might already be wondering: What's up with that first record?

Row 0 is what we call a **rollup record**, or summary record. It's often useful when displaying tables to humans. The **granularity** of record 0 (Totals) vs the rest of the records (States) is different.

<br/>

Okay, EDA step two. How was the rollup record aggregated?

Let's check if Total TB cases is the sum of all state TB cases. If we sum over all rows, we should get **2x** the total cases in each of our TB cases by year (why?).

In [70]:
# Apply sum to tb DataFrame
tuberculosis_data ["Total_Cases"] = tuberculosis_data[['TB_cases(2019)','TB_cases(2020)','TB_cases(2021)']].sum(axis=1)
tuberculosis_data ["Total_incidence"] = tuberculosis_data[['TB_inci(2019)','TB_inci(2020)','TB_inci(2021)']].sum(axis=1)


In [71]:
tuberculosis_data.head()


,States,TB_cases(2019),TB_cases(2020),TB_cases(2021),TB_inci(2019),TB_inci(2020),TB_inci(2021),Total_Cases,Total_incidence
0,Total,8900,7173,7860,2.71,2.16,2.37,23933,7.24
1,Alabama,87,72,92,1.77,1.43,1.83,251,5.03
2,Alaska,58,58,58,7.91,7.92,7.92,174,23.75
3,Arizona,183,136,129,2.51,1.89,1.77,448,6.17
4,Arkansas,64,59,69,2.12,1.96,2.28,192,6.36


<br/>

Whoa, what's going on? Check out the column types:

In [72]:
# Find datatype for each column
tuberculosis_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   States           52 non-null     object 
 1   TB_cases(2019)   52 non-null     int64  
 2   TB_cases(2020)   52 non-null     int64  
 3   TB_cases(2021)   52 non-null     int64  
 4   TB_inci(2019)    52 non-null     float64
 5   TB_inci(2020)    52 non-null     float64
 6   TB_inci(2021)    52 non-null     float64
 7   Total_Cases      52 non-null     int64  
 8   Total_incidence  52 non-null     float64
dtypes: float64(4), int64(4), object(1)
memory usage: 3.8+ KB


<br/>

Looks like those commas are causing all TB cases to be read as the `object` datatype, or **storage type** (close to the Python string datatype), so pandas is concatenating strings instead of adding integers.

<br/>

Fortunately `read_csv` also has a `thousands` parameter (for what it's worth, I didn't know this beforehand--I [googled](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) this):

In [75]:
# improve readability: chaining method calls with outer parentheses/line breaks
tuberculosis_data = pd.read_csv("/content/cdc_tuberculosis.csv",thousands=',',header=1)

tuberculosis_data.columns = cols

tuberculosis_data.head(5)

,States,TB_cases(2019),TB_cases(2020),TB_cases(2021),TB_inci(2019),TB_inci(2020),TB_inci(2021)
0,Total,8900,7173,7860,2.71,2.16,2.37
1,Alabama,87,72,92,1.77,1.43,1.83
2,Alaska,58,58,58,7.91,7.92,7.92
3,Arizona,183,136,129,2.51,1.89,1.77
4,Arkansas,64,59,69,2.12,1.96,2.28


In [108]:
# now apply sum
# Answer Here
tuberculosis_data ["Total_Cases"] = tuberculosis_data [['TB_cases(2019)','TB_cases(2020)','TB_cases(2021)']].sum(axis=1)
tuberculosis_data ["Total_incidence"] = tuberculosis_data [['TB_inci(2019)','TB_inci(2020)','TB_inci(2021)']].sum(axis=1)

In [109]:
tuberculosis_data.head()

,States,TB_cases(2019),TB_cases(2020),TB_cases(2021),TB_inci(2019),TB_inci(2020),TB_inci(2021),Total_Cases,Total_incidence
1,Alabama,87,72,92,1.77,1.43,1.83,251,5.03
2,Alaska,58,58,58,7.91,7.92,7.92,174,23.75
3,Arizona,183,136,129,2.51,1.89,1.77,448,6.17
4,Arkansas,64,59,69,2.12,1.96,2.28,192,6.36
5,California,2111,1706,1750,5.35,4.32,4.46,5567,14.13


The Total TB cases look right. Phew!

(We'll leave it to your own EDA to figure out how the TB incidence "Totals" were aggregated.)

Let's just look at the records with **state-level granularity**:

What do each of these values represent? Why?

To the lecture!


# Gather Census Data

U.S. Census population estimates [source](https://www.census.gov/data/tables/time-series/demo/popest/2010s-state-total.html) (2019), [source](https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-total.html) (2020-2021).

Running the below cells cleans the data. We encourage you to closely explore the CSV and study these lines after lecture...

There are a few new methods here:
* `df.convert_dtypes()` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.convert_dtypes.html)) conveniently converts all float dtypes into ints and is out of scope for the class.
* `df.drop_na()` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html)) will be explained in more detail next time.

In [85]:
# Load 2010s census data
census_data = pd.read_csv(r"/content/nst-est2019-01.csv",header=3,thousands=",")
census_data.head(5)



,Unnamed: 0,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,United States,308745538.0,308758105.0,309321666.0,311556874.0,313830990.0,315993715.0,318301008.0,320635163.0,322941311.0,324985539.0,326687501.0,328239523.0
1,Northeast,55317240.0,55318443.0,55380134.0,55604223.0,55775216.0,55901806.0,56006011.0,56034684.0,56042330.0,56059240.0,56046620.0,55982803.0
2,Midwest,66927001.0,66929725.0,66974416.0,67157800.0,67336743.0,67560379.0,67745167.0,67860583.0,67987540.0,68126781.0,68236628.0,68329004.0
3,South,114555744.0,114563030.0,114866680.0,116006522.0,117241208.0,118364400.0,119624037.0,120997341.0,122351760.0,123542189.0,124569433.0,125580448.0
4,West,71945553.0,71946907.0,72100436.0,72788329.0,73477823.0,74167130.0,74925793.0,75742555.0,76559681.0,77257329.0,77834820.0,78347268.0


In [86]:
census_data2 = pd.read_csv(r"/content/NST-EST2022-POP.csv",header=3,thousands=",")
census_data2.head()

census_data2.rename(columns={"Unnamed: 0": "Geographic Area"},inplace=True)

# Apply some EDA

Drop the column of Estiamte Base

Rename Unnamed: 0 as 'Geographic Area'

"smart" converting of columns, use at your own risk

.dropna() to drop records with NaN

You can also suggest any change that can be helpful for EDA



In [87]:

# with pd.option_context('display.min_rows', 30): # shows more rows
#     display(census_2010s_df)
census_data = (
    census_data
    .drop(columns=["Estimates Base"])
    .rename(columns={"Unnamed: 0": "Geographic Area"})
    .convert_dtypes()
    .dropna()
)




In [88]:
census_data2.dropna(inplace=True)

In [89]:
# census 2020s data
census_data2.head(5)

,Geographic Area,Unnamed: 1,2020,2021,2022
0,United States,331449520.0,331511512.0,332031554.0,333287557.0
1,Northeast,57609156.0,57448898.0,57259257.0,57040406.0
2,Midwest,68985537.0,68961043.0,68836505.0,68787595.0
3,South,126266262.0,126450613.0,127346029.0,128716192.0
4,West,78588565.0,78650958.0,78589763.0,78743364.0


In [90]:
census_data["Geographic Area"].str.len().head(2)

0    13
1     9
Name: Geographic Area, dtype: Int64

In [91]:
census_data["Geographic Area"] = census_data["Geographic Area"].str.strip('.')
census_data2["Geographic Area"] = census_data2["Geographic Area"].str.strip('.')

In [92]:
# perform EDA Here
print(census_data.columns)
print(tuberculosis_data.columns)
print(census_data2.columns)

Index(['Geographic Area', 'Census', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019'],
      dtype='object')
Index(['States', 'TB_cases(2019)', 'TB_cases(2020)', 'TB_cases(2021)',
       'TB_inci(2019)', 'TB_inci(2020)', 'TB_inci(2021)', 'Total_Cases',
       'Total_incidence'],
      dtype='object')
Index(['Geographic Area', 'Unnamed: 1', '2020', '2021', '2022'], dtype='object')


<br/><br/>

---

# Join Data (Merge DataFrames)

Time to `merge`! Here I use the DataFrame method `df1.merge(right=df2, ...)` on DataFrame `df1` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html)). Contrast this with the function `pd.merge(left=df1, right=df2, ...)` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.merge.html?highlight=pandas%20merge#pandas.merge)). Feel free to use either.

In [93]:
# merge TB dataframe with two US census dataframes
merged_tb_dataframe = tuberculosis_data.merge(right=census_data,
           left_on='States', right_on="Geographic Area").merge(right=census_data2,
           left_on='States', right_on="Geographic Area")

In [94]:
pd.set_option('display.max_columns',33)
merged_tb_dataframe.head()

,States,TB_cases(2019),TB_cases(2020),TB_cases(2021),TB_inci(2019),TB_inci(2020),TB_inci(2021),Total_Cases,Total_incidence,Geographic Area_x,Census,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Geographic Area_y,Unnamed: 1,2020,2021,2022
0,Alabama,87,72,92,1.77,1.43,1.83,251,5.03,Alabama,4779736,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185,Alabama,5024356.0,5031362.0,5049846.0,5074296.0
1,Alaska,58,58,58,7.91,7.92,7.92,174,23.75,Alaska,710231,713910,722128,730443,737068,736283,737498,741456,739700,735139,731545,Alaska,733378.0,732923.0,734182.0,733583.0
2,Arizona,183,136,129,2.51,1.89,1.77,448,6.17,Arizona,6392017,6407172,6472643,6554978,6632764,6730413,6829676,6941072,7044008,7158024,7278717,Arizona,7151507.0,7179943.0,7264877.0,7359197.0
3,Arkansas,64,59,69,2.12,1.96,2.28,192,6.36,Arkansas,2915918,2921964,2940667,2952164,2959400,2967392,2978048,2989918,3001345,3009733,3017804,Arkansas,3011555.0,3014195.0,3028122.0,3045637.0
4,California,2111,1706,1750,5.35,4.32,4.46,5567,14.13,California,37253956,37319502,37638369,37948800,38260787,38596972,38918045,39167117,39358497,39461588,39512223,California,39538245.0,39501653.0,39142991.0,39029342.0


This is a little unwieldy. We could either drop the unneeded columns now, or just merge on smaller census DataFrames. Let's do the latter.

In [100]:
# try merging again, but cleaner this time
merged = (tuberculosis_data.merge(right=census_data[["Geographic Area", "2019"]],
           left_on="States", right_on="Geographic Area").drop(columns="Geographic Area").merge(right=census_data2,
           left_on="States", right_on="Geographic Area").drop(columns=["Geographic Area"])
)
merged.head()

,States,TB_cases(2019),TB_cases(2020),TB_cases(2021),TB_inci(2019),TB_inci(2020),TB_inci(2021),Total_Cases,Total_incidence,2019,Unnamed: 1,2020,2021,2022
0,Alabama,87,72,92,1.77,1.43,1.83,251,5.03,4903185,5024356.0,5031362.0,5049846.0,5074296.0
1,Alaska,58,58,58,7.91,7.92,7.92,174,23.75,731545,733378.0,732923.0,734182.0,733583.0
2,Arizona,183,136,129,2.51,1.89,1.77,448,6.17,7278717,7151507.0,7179943.0,7264877.0,7359197.0
3,Arkansas,64,59,69,2.12,1.96,2.28,192,6.36,3017804,3011555.0,3014195.0,3028122.0,3045637.0
4,California,2111,1706,1750,5.35,4.32,4.46,5567,14.13,39512223,39538245.0,39501653.0,39142991.0,39029342.0


In [101]:
merged.drop(columns=["Total_Cases",	"Total_incidence"],inplace=True)


## Reproduce incidence

Let's recompute incidence to make sure we know where the original CDC numbers came from.

From the [CDC report](https://www.cdc.gov/mmwr/volumes/71/wr/mm7112a1.htm?s_cid=mm7112a1_w#T1_down): TB incidence is computed as “Cases per 100,000 persons using mid-year population estimates from the U.S. Census Bureau.”

If we define a group as 100,000 people, then we can compute the TB incidence for a given state population as

$$\text{TB incidence} = \frac{\text{# TB cases in population}}{\text{# groups in population}} = \frac{\text{# TB cases in population}}{\text{population}/100000} $$

$$= \frac{\text{# TB cases in population}}{\text{population}} \times 100000$$

Let's try this for 2019:

In [102]:
merged["recompute incidence 2019"] = merged["TB_inci(2019)"]/merged['TB_inci(2019)']*100000

In [103]:
merged.head(5)

,States,TB_cases(2019),TB_cases(2020),TB_cases(2021),TB_inci(2019),TB_inci(2020),TB_inci(2021),2019,Unnamed: 1,2020,2021,2022,recompute incidence 2019
0,Alabama,87,72,92,1.77,1.43,1.83,4903185,5024356.0,5031362.0,5049846.0,5074296.0,100000.0
1,Alaska,58,58,58,7.91,7.92,7.92,731545,733378.0,732923.0,734182.0,733583.0,100000.0
2,Arizona,183,136,129,2.51,1.89,1.77,7278717,7151507.0,7179943.0,7264877.0,7359197.0,100000.0
3,Arkansas,64,59,69,2.12,1.96,2.28,3017804,3011555.0,3014195.0,3028122.0,3045637.0,100000.0
4,California,2111,1706,1750,5.35,4.32,4.46,39512223,39538245.0,39501653.0,39142991.0,39029342.0,100000.0


Awesome!!!

Let's use a for-loop and Python format strings to compute TB incidence for all years. Python f-strings are just used for the purposes of this demo, but they're handy to know when you explore data beyond this course ([Python documentation](https://docs.python.org/3/tutorial/inputoutput.html)).

In [104]:
# recompute incidence for all years
recompute_years = ["TB_cases(2019)","TB_cases(2020)","TB_cases(2021)"]
for year in recompute_years:
    merged[f"recompute incidence {year}"] = merged[f"{year}"]/merged['2019']*100000
merged.head(5)

,States,TB_cases(2019),TB_cases(2020),TB_cases(2021),TB_inci(2019),TB_inci(2020),TB_inci(2021),2019,Unnamed: 1,2020,2021,2022,recompute incidence 2019,recompute incidence TB_cases(2019),recompute incidence TB_cases(2020),recompute incidence TB_cases(2021)
0,Alabama,87,72,92,1.77,1.43,1.83,4903185,5024356.0,5031362.0,5049846.0,5074296.0,100000.0,1.774357,1.468433,1.876331
1,Alaska,58,58,58,7.91,7.92,7.92,731545,733378.0,732923.0,734182.0,733583.0,100000.0,7.928425,7.928425,7.928425
2,Arizona,183,136,129,2.51,1.89,1.77,7278717,7151507.0,7179943.0,7264877.0,7359197.0,100000.0,2.514179,1.868461,1.77229
3,Arkansas,64,59,69,2.12,1.96,2.28,3017804,3011555.0,3014195.0,3028122.0,3045637.0,100000.0,2.120747,1.955064,2.286431
4,California,2111,1706,1750,5.35,4.32,4.46,39512223,39538245.0,39501653.0,39142991.0,39029342.0,100000.0,5.342651,4.317651,4.429009


These numbers look pretty close!!! There are a few errors in the hundredths place, particularly in 2021. It may be useful to further explore reasons behind this discrepancy. We'll leave it to you!